# Movielens 영화 추천

1. 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.

2. 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.

3. 별점을 시청횟수로 해석해서 생각하겠습니다.

4. 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.

In [1]:
! wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

--2021-02-16 00:34:14--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  9.10MB/s    in 0.6s    

2021-02-16 00:34:15 (9.10 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]



In [2]:
! mv ml-1m.zip ~/aiffel/recommendata_iu/data

In [3]:
! cd ~/aiffel/recommendata_iu/data

In [4]:
! unzip ml-1m.zip

unzip:  cannot find or open ml-1m.zip, ml-1m.zip.zip or ml-1m.zip.ZIP.


In [5]:
cd ~/aiffel/recommendata_iu/data

/home/aiffel-dj43/aiffel/recommendata_iu/data


In [6]:
! pwd

/home/aiffel-dj43/aiffel/recommendata_iu/data


In [1]:
! unzip ml-1m.zip

unzip:  cannot find or open ml-1m.zip, ml-1m.zip.zip or ml-1m.zip.ZIP.


아까도 왜 명령어가 안먹나 했는데 느낌표를 붙이면 안됐던거다.

## 1) 데이터 준비와 전처리

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
ratings = ratings[ratings['rating']>=3] #3점 이상 남기기
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
ratings.rename(columns={'rating':'count'}, inplace=True) #ratings 부분을 count로 바꾸기

In [4]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat' # 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## 2) 분석

### ratings에 있는 유니크한 영화 개수/사용자 수

In [6]:
ratings['movie_id'].nunique()

3628

In [7]:
ratings['user_id'].nunique()

6039

### 인기 영화 탑 30

In [8]:
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

## 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가

사실 데이터를 확인하는 방법에 대해 잘 몰랐는데 캐글을 진행하면서 팀원들 중 한 분이 엑셀을 이용해서 데이터를 대략적으로 확인하는 방법에 대해 알려주셨다.  

물론 이번 파일은 정상적으로 열리지 않아 대략적으로만 볼 수 있었는데 어찌됐던 엑셀을 통해 데이터를 파악했다.  

다만, 코딩을 이용해서 어떻게 하는 방법이 있는지에 대해서는 아직 이해가 부족한 것 같다. 그래서 내가 선호하는 영화 5개는 못 골랐고 그냥 위에 보이는 5개를 선택했다.

In [24]:
my_favorite = [2137, 1099, 897, 717, 545]

my_movielist = pd.DataFrame({'user_id': ['hong']*5, 'movie_id': my_favorite, 'count':[5]*5})

if not ratings.isin({'user_id':['hong']})['user_id'].any():  
    ratings = ratings.append(my_movielist)                           

ratings.tail(10)       

,user_id,movie_id,count,timestamp
0,6039,520,5,NaN
1,6039,1929,5,NaN
2,6039,1526,5,NaN
3,6039,2606,5,NaN
4,6039,3455,5,NaN
0,hong,2137,5,NaN
1,hong,1099,5,NaN
2,hong,897,5,NaN
3,hong,717,5,NaN
4,hong,545,5,NaN


In [25]:
#unique 사용해 고유 유저와 영화 찾아내기
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

In [26]:
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [27]:
print(user_to_idx['hong']) 
print(movie_to_idx[367])

6040
367


In [28]:
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data    
else:
    print('user_id column indexing Fail!!')

temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('artist column indexing Fail!!')

ratings

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,count,timestamp
0,0,0,5,978300760.0
1,0,1,3,978302109.0
2,0,2,3,978301968.0
3,0,3,4,978300275.0
4,0,4,5,978824291.0
...,...,...,...,...
0,6040,2137,5,NaN
1,6040,1099,5,NaN
2,6040,897,5,NaN
3,6040,717,5,NaN


## 4) CSR matrix를 직접 만들기

In [29]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6041x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836488 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련

사실 이 부분에서 어떻게 모델을 구성하라는 것인지 사실 이해를 못했으나 눈치껏 노드에 MF 모델이 있으니 그걸 사용해보겠습니다.

In [30]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [31]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [32]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6041 sparse matrix of type '<class 'numpy.int64'>'
	with 836488 stored elements in Compressed Sparse Column format>

In [33]:
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악하기

In [34]:
hong, magica = user_to_idx['hong'], movie_to_idx[545]
hong_vector, magica_vector = als_model.user_factors[hong], als_model.item_factors[magica]

print('슝=3')

슝=3


원래는 betty blowtorch, 2137로 하려 했으나 띄어쓰기 때문인지 invalid syntax가 나와 우선 magica 545로 바꿔봤다. 2137으로 했을때 위에처럼 Nan이 많이 나오지 않았는데 545로 하니 Nan이 많이 나왔다.

In [35]:
hong_vector

array([ 0.02621982,  0.2861235 ,  0.5058485 , -0.2868243 ,  0.45004666,
       -0.07269838, -0.01819208, -0.06827383, -0.55487823,  0.14044607,
       -0.22053562,  0.17966004, -0.05193571,  0.21894047, -0.12872495,
       -0.00537793,  0.5689115 , -0.29613993,  0.39476097, -0.05342838,
        0.09725819,  0.67801005, -0.53308904,  0.67795265, -0.33217543,
       -0.19318473, -0.40535888, -0.1841298 ,  0.1704233 ,  0.48021257,
        0.2567539 ,  0.03963212, -0.25189248, -0.12880218, -0.10316706,
        0.12865892, -0.28571153,  0.3457199 ,  0.04960471, -0.2861416 ,
        0.25848043,  0.6489091 , -0.22823246,  0.02841105, -0.13744944,
        0.73574686, -0.31803247, -0.11621652, -0.12030667,  0.33885714,
       -0.41058493,  0.7914482 , -0.23549406,  0.40118262, -0.83786863,
       -0.6453801 ,  0.4691003 , -0.07037676, -0.5051146 , -0.17766163,
       -0.44884622, -0.06449979,  0.07444172,  0.6684728 ,  0.2900348 ,
       -0.49363485,  0.19489765,  0.18792722,  0.11055525,  0.06

In [36]:
magica_vector

array([ 5.1821815e-03,  1.0587677e-02,  1.0980408e-02, -2.0435459e-03,
        9.8680761e-03,  2.8909938e-03,  2.9501617e-02,  4.4961413e-03,
       -8.6380728e-03,  2.3264878e-03,  6.9822147e-03, -8.1929495e-04,
       -8.8036386e-03,  8.6417738e-03, -1.4593807e-02, -3.6393504e-03,
        1.0237023e-02, -1.4893112e-03,  4.1667516e-03, -3.8204915e-03,
       -1.7019799e-02,  1.6945107e-02, -2.9382289e-03,  4.0160775e-02,
       -1.6998356e-02,  6.1255302e-03, -2.5898437e-03,  6.7241126e-05,
        1.1355064e-02,  8.1148883e-03,  2.2076868e-02, -8.0410467e-04,
       -2.2236321e-03,  2.5017962e-03, -4.6716034e-03,  1.3620301e-02,
        1.5527709e-02,  1.1631965e-02, -4.9525528e-04,  5.3904615e-03,
        6.8854014e-03,  6.4680115e-03,  1.2418557e-02,  1.5752036e-02,
        8.3405096e-03,  2.8047983e-02,  8.6836992e-03, -9.0351328e-03,
       -6.2134606e-04,  2.7832214e-02, -2.1610877e-02,  2.7658751e-02,
        6.3724327e-03,  1.1571410e-02,  1.3748418e-02, -9.6538980e-03,
      

In [37]:
np.dot(hong_vector, magica_vector)

0.32582128

노드 설명을 보면 1이 기준이라 0.49도 낮다고 했는데 0.32가 나왔다. 아마 랜덤으로 영화를 뽑아서 그런 것 같은데 지금은 시간이 없으니 일단 제출해놓고 데이터 분석에 대해서 좀더 알아봐야겠다.

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받기

In [38]:
movie_id = movie_to_idx[1]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(1, 1.0),
 (284, 0.642957),
 (35, 0.6366633),
 (951, 0.5846794),
 (329, 0.5602242),
 (1986, 0.55007434),
 (1513, 0.53931993),
 (34, 0.5305506),
 (705, 0.52976316),
 (25, 0.5191251),
 (947, 0.51900524),
 (32, 0.51443714),
 (1556, 0.50660837),
 (3007, 0.5034905),
 (609, 0.5027095)]

In [39]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

[1, 284, 35, 951, 329, 1986, 1513, 34, 705, 25, 947, 32, 1556, 3007, 609]

In [40]:
def get_similar_movie(movie_name: int):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

print("슝=3")

슝=3


In [43]:
#movie_id 1번은 '토이 스토리'
get_similar_movie(1)

[1, 284, 35, 951, 329, 1986, 1513, 34, 705, 25]

## 8) 내가 가장 좋아할 만한 영화들을 추천받기

In [44]:
user = user_to_idx['hong']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(755, 0.24279442),
 (461, 0.21760005),
 (458, 0.20712234),
 (1853, 0.1968343),
 (1814, 0.19587734),
 (513, 0.18020287),
 (1887, 0.16841874),
 (909, 0.1680873),
 (596, 0.16415197),
 (688, 0.15858173),
 (812, 0.15653901),
 (463, 0.15307602),
 (173, 0.15170863),
 (486, 0.15132143),
 (669, 0.1470106),
 (675, 0.14575823),
 (1465, 0.1454271),
 (60, 0.1453163),
 (474, 0.14492722),
 (1130, 0.14387307)]

In [45]:
[idx_to_movie[i[0]] for i in movie_recommended]

[755,
 461,
 458,
 1853,
 1814,
 513,
 1887,
 909,
 596,
 688,
 812,
 463,
 173,
 486,
 669,
 675,
 1465,
 60,
 474,
 1130]

In [46]:
glass_bottom_boat = movie_to_idx[3144]
explain = als_model.explain(user, csr_data, itemid=glass_bottom_boat)

In [47]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[(2137, 0.002161084584985504),
 (717, -0.00021323441818190037),
 (897, -0.00041779800914732346),
 (545, -0.0038486351280029077),
 (1099, -0.004836982572184133)]

### 캐글을 진행하면서 데이터 이해가 매우 중요함을 느꼈다. 특히 팀원들과 데이터 이해하는 것에만 거의 캐글 진행 시간의 50% 이상을 사용했는데 그만큼 중요한 데이터 이해를 지금까지 소흘히 했다는 것을 반성한다.   

### 하지만 이번 EXPLORATION에서 데이터에 대한 이해를 많이 하지 못했다. 캐글에서 했듯이 엑셀을 통해서 전체 데이터를 확인해보려고 했는데 파일이 너무 커서 그런지 자꾸 키는데 오류가 났다. 그래서 내가 선호하는 영화 id를 찾지 못하고 랜덤한 id를 입력했다 .그래서 그런지 수치도 낮게 나오고 이 모델이 정확하게 내가 선호하는 영화를 추천해주건지 막 와닿지는 않는다.  

### 지금 약간 시간이 쫓겨서 일단 프로젝트를 제출하지만 좀더 시간을 들여서 데이터 이해를 어떻게 해야할까에 대한 고민을 해보고 팀원들에게 조언을 구해야겠다.